<a href="https://colab.research.google.com/github/hajerhajjajifahem24/Repo-2017/blob/master/LSTMmodel_analysis_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import tensorflow as tf

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data = pd.read_csv('googleplaystore_user_reviews.csv')

In [ ]:
data

In [ ]:
data = data[['Translated_Review','Sentiment']]

In [ ]:
data=data.dropna()

In [ ]:
data

In [ ]:
data = data[data.Sentiment != "Neutral"]
data['Translated_Review'] = data['Translated_Review'].apply(lambda x: x.lower())
data['Translated_Review'] = data['Translated_Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sentiment'] == 'Positive'].size)
print(data[ data['Sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Translated_Review'].values)
X = tokenizer.texts_to_sequences(data['Translated_Review'].values)
X = pad_sequences(X)

In [ ]:
X

In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
Y_train

In [ ]:
X.shape[1]

In [ ]:
model=Sequential()
model.add(Embedding(15000,512,mask_zero=True))
model.add(LSTM(512,dropout=0.1, recurrent_dropout=0.1,return_sequences=True))
model.add(LSTM(256,dropout=0.1, recurrent_dropout=0.1,return_sequences=False))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=( X_test, Y_test), epochs=4, batch_size=1000, verbose=1)